In [1]:
import numpy as np
from graphviz import Digraph

In [2]:
class NFA:
    
    def __init__(self, re_str):
        self.re_str = re_str
        self.postfix_expression = ''
        self.states_number = 0
        self.nfa_graph = {}
        self.b_e = None
    
    def priority(self, operator):
        if operator == "*":
            return 3
        elif operator == ".":
            return 2
        elif operator == "|":
            return 1
        else:
            return 0
    def gen_postfix_expression(self):
        re_with_connector = ''
        for i in range(len(self.re_str) - 1):
            re_with_connector += self.re_str[i]
            if (self.re_str[i] != "(" and self.re_str[i+1] != ")" and self.re_str[i] != "|"
                and self.re_str[i+1] != "|" and self.re_str[i+1] != "*"):
                re_with_connector += "."
        re_with_connector += self.re_str[-1]
        ops = []
        for item in re_with_connector:
            if (item == "("):
                ops.append(item)
            elif (item == ")"):
                while (ops[-1] != "("):
                    self.postfix_expression += ops.pop(-1)
                ops.pop(-1)
            elif (item == "*" or item == "." or item == "|"):
                while (len(ops) > 0):
                    if (self.priority(item) <= self.priority(ops[-1])):
                        self.postfix_expression += ops.pop(-1)
                    else:
                        break
                ops.append(item)
            else:
                self.postfix_expression += item
        while(len(ops) > 0):
            self.postfix_expression += ops.pop(-1)

    def gen_new_state(self):
        self.states_number += 1

        return self.states_number
   
    def add_edge_for_state(self, state, edge):
        keys = list(self.nfa_graph.keys())
        if state in keys:
            self.nfa_graph[state].append(edge)
        else:
            self.nfa_graph[state] = [edge]

    def gen_nfa_from_pe(self):
        states = []
        for item in self.postfix_expression:
            if(item != "*" and item != "." and item != "|"):
                begin_state = self.gen_new_state()
                end_state = self.gen_new_state()
                states.append([begin_state, end_state])
                self.add_edge_for_state(begin_state, [end_state, item])
                #continue

            elif(item == "*"):
                origin_item = states.pop(-1)
                begin_state = self.gen_new_state()
                end_state = self.gen_new_state()
                states.append([begin_state, end_state])
                self.add_edge_for_state(begin_state, [origin_item[0], 'epsilon'])
                self.add_edge_for_state(begin_state, [end_state, 'epsilon'])   
                self.add_edge_for_state(origin_item[1], [end_state, 'epsilon'])
                self.add_edge_for_state(origin_item[1], [origin_item[0], 'epsilon'])

            elif(item == "."):
                right = states[-1]
                states.pop(-1)
                left = states[-1]
                states.pop(-1)
                states.append([left[0], right[1]])
                self.add_edge_for_state(left[1], [right[0], 'epsilon']) 

            elif(item == "|"):
                down = states.pop(-1)
                up = states.pop(-1)
                begin_state = self.gen_new_state()
                end_state = self.gen_new_state()
                states.append([begin_state, end_state])
                self.add_edge_for_state(begin_state, [up[0], 'epsilon'])
                self.add_edge_for_state(begin_state, [down[0], 'epsilon'])
                self.add_edge_for_state(up[1], [end_state, 'epsilon'])
                self.add_edge_for_state(down[1], [end_state, 'epsilon'])
                
        self.b_e = states[-1]         
        
        return 0
            
    def visualize_nfa(self):
        dot = Digraph("re2NFA", format='png')
        for key, value in self.nfa_graph.items():
            dot.node(name=str(key))
            for sub in value:
                if sub[0] == self.b_e[1]:
                    dot.node(name=str(sub[0]), shape='doublecircle')
                    if (sub[1] == "epsilon" and key != self.b_e[0]):
                        dot.node(name=str(key), shape='doublecircle')
                else:
                    dot.node(name=str(sub[0]))
                dot.edge(str(key), str(sub[0]), label=sub[1])
        dot.view()
 
        return 0

if __name__ == "__main__":
    test_case=['abc', 'a|b', 'a*', '(a|b)*', '(abc)*', '(a|b)*c', '(a|b)c*']
    nfa = NFA(test_case[2])
    nfa.gen_postfix_expression()
    #print(nfa.postfix_expression)
    nfa.gen_nfa_from_pe()
    #print(nfa.b_e)
    #print(nfa.nfa_graph)
    nfa.visualize_nfa()